<a href="https://colab.research.google.com/github/Angel-Maravilla/FS-GRIND/blob/main/STOPSIGNDETECTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip -q install pycocotools tqdm gradio scikit-learn  #quiet install, pycocotools reads coco annotations, tqdm for progress bars
#Imports
import os, json, random, io
from pathlib import Path
from PIL import Image, ImageDraw
from tqdm.auto import tqdm           #progress bar, clean file paths, rand and json

import torch
import torch.nn as nn #torch.nn for neural network layers and losses
from torch.utils.data import Dataset, DataLoader #Data set and loader help

import torchvision
from torchvision import transforms #to recrop and normalize images
from torchvision.models import resnet18, ResNet18_Weights #pretrained weights and classifier
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights #pretrained detector

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report  #evaluation
import gradio as gr  #easy drag and drop app
from google.colab import files #upload fules manually

##COCO download and path set up
DATA_ROOT = Path("/content/coco") #coco
IMG_SPLIT = "val2017"   #image splt useed
ANN_YEAR  = "2017"  #annotation year
DATA_ROOT.mkdir(parents=True, exist_ok=True) #make folder

urls = {
    "val2017":   "http://images.cocodataset.org/zips/val2017.zip",
    "train2017": "http://images.cocodataset.org/zips/train2017.zip",
    "annotations": f"http://images.cocodataset.org/annotations/annotations_trainval{ANN_YEAR}.zip", #download links for images and annotations
}

def download_and_unzip(url, dest_dir=DATA_ROOT):
    fname = url.split("/")[-1]
    zip_path = dest_dir / fname
    if not zip_path.exists():
        !wget -q "{url}" -P "{dest_dir}"
    !unzip -q -o "{zip_path}" -d "{dest_dir}"   #grab filename, check if downloaded, if not wget, unzip to destination

download_and_unzip(urls[IMG_SPLIT])
download_and_unzip(urls["annotations"])  #download img split and annotations

img_dir  = DATA_ROOT / IMG_SPLIT
ann_file = DATA_ROOT / "annotations" / f"instances_{IMG_SPLIT.replace('2017','')}{ANN_YEAR}.json"
print("Images dir:", img_dir)
print("Ann file:", ann_file)    ## set paths for images and annotate json, print

##Build pos and neg image list for stop sign
from pycocotools.coco import COCO
coco = COCO(str(ann_file))   #load coco annot to query categories and images
cats = coco.loadCats(coco.getCatIds())
coco_classes = {c["name"]: c["id"] for c in cats} # get categories and make name id dictionary
assert "stop sign" in coco_classes, "COCO must contain 'stop sign'."
target_catId = coco_classes["stop sign"]   #STOP SIGN EXIST get numeric category id

imgIds_pos = coco.getImgIds(catIds=[target_catId])
imgIds_all = coco.getImgIds()
imgIds_neg = [i for i in imgIds_all if i not in set(imgIds_pos)]  #neg or pos images

random.seed(0)
neg_keep      = min(len(imgIds_neg), len(imgIds_pos))
imgIds_neg_ds = random.sample(imgIds_neg, neg_keep)  #Fixed seed keeps it around 50/50

pairs = [(i,1) for i in imgIds_pos] + [(i,0) for i in imgIds_neg_ds]
random.shuffle(pairs)
print("Counts | pos, neg, total =", len(imgIds_pos), len(imgIds_neg_ds), len(pairs)) #shuffle pairs id, lavel

##dataset and loader tranformation
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]  #expected nromalization of pretrained

train_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),   # resize, crop, eugment and convert to tensor
])

val_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),   #validates
])

class COCOPresenceDataset(Dataset):
    def __init__(self, coco, image_dir, id_label_pairs, transform):
        self.coco = coco
        self.image_dir = Path(image_dir)
        self.items = id_label_pairs
        self.transform = transform
        self._cache = {}  ###coco index, image dir, id,label, transformatio and cache mapping

    def __len__(self): return len(self.items)

    def __getitem__(self, idx):
        img_id, label = self.items[idx]
        if img_id not in self._cache:
            info = self.coco.loadImgs([img_id])[0]
            self._cache[img_id] = info["file_name"]
        fp = self.image_dir / self._cache[img_id]
        img = Image.open(fp).convert("RGB")
        img = self.transform(img)
        return img, torch.tensor([label], dtype=torch.float32)  #grab an item, cache it open it, tranform then return is as a tensor float for loss

#split
n = len(pairs)
val_n = max(1, int(0.15 * n))
val_pairs = pairs[:val_n]
train_pairs = pairs[val_n:]  #use 15%for validation the rest for training

ds_train = COCOPresenceDataset(coco, img_dir, train_pairs, train_tf)
ds_val   = COCOPresenceDataset(coco, img_dir, val_pairs,   val_tf)   #train and val datasets

#multiproccess fix
dl_train = DataLoader(ds_train, batch_size=64, shuffle=True,  num_workers=0, pin_memory=False)
dl_val   = DataLoader(ds_val,   batch_size=64, shuffle=False, num_workers=0, pin_memory=False) #data loaders, 64imgs a batch, shuffle

print("Dataset sizes | train, val =", len(ds_train), len(ds_val))

#Model & training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #pick GPU if available else use CPU.
print("Device:", device)

#Uses ResNet-18 for the weights and the model
weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)
model.fc = nn.Linear(model.fc.in_features, 1)  # 1 logit for binary presence
model.to(device)

criterion = nn.BCEWithLogitsLoss() #define loss for binary classification that expects raw logits
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4) #sets up Adam with learning rate and weight decay



#Run one full pass
def run_epoch(loader, train=True):
  # set mode (affects dropout/batchnorm)
    model.train(train)
    total_loss, total, correct = 0.0, 0, 0
    # only track gradients in training to save memory/time
    with torch.set_grad_enabled(train):
        for x, y in tqdm(loader, leave=False):
           # move batch to device
            x, y = x.to(device), y.to(device)
            if train:
                optimizer.zero_grad(set_to_none=True) #clear old grads
                logits = model(x)
                loss = criterion(logits, y) #compute loss
                loss.backward()  # backprop
                optimizer.step()  #update weight
            else:
                logits = model(x)
                loss = criterion(logits, y)

            total_loss += loss.item() * x.size(0)
            total += x.size(0)
            preds = (torch.sigmoid(logits) >= 0.5).float()
            correct += (preds == y).all(dim=1).sum().item()
    # average loss over all samples, and accuracy
    return total_loss / total, correct / total

best_val = float("inf")
best_state = None
patience, bad = 5, 0
EPOCHS = 5  # try 10–15 with IMG_SPLIT="train2017"

for e in range(1, EPOCHS + 1):
    tr_loss, tr_acc = run_epoch(dl_train, train=True)
    va_loss, va_acc = run_epoch(dl_val,   train=False)
    print(f"Epoch {e:02d} | train {tr_loss:.4f}/{tr_acc:.3f} | val {va_loss:.4f}/{va_acc:.3f}")
    if va_loss < best_val - 1e-4:
        best_val, bad = va_loss, 0
        best_state = {k: v.cpu() for k, v in model.state_dict().items()}
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping.")
            break

if best_state:
    model.load_state_dict(best_state)
    model.to(device)



def eval_loader(loader, thresh=0.5):
    model.eval() #model in evaluation mode
    probs, ys = [], []
    with torch.no_grad():  #faster, uses less memory—good for eval
        for x, y in loader:
            x = x.to(device) # moves batch to same device
            #runs the model and gets raw scores
            p = torch.sigmoid(model(x)).cpu().numpy().ravel()
            probs.append(p); ys.append(y.numpy().ravel())
    probs = np.concatenate(probs); ys = np.concatenate(ys)
    preds = (probs >= thresh).astype(int)

    # report
    print("Confusion matrix [TN FP; FN TP]:")
    print(confusion_matrix(ys, preds))
    print(classification_report(ys, preds, digits=3))
    return probs, ys # keep raw probs + true labels for later

_ = eval_loader(dl_val, thresh=0.5)


infer_tf = val_tf  # deterministic eval preprocessing

#Run the classifier on one image
def predict_image(img: Image.Image, threshold=0.5):
    model.eval()
    x = infer_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        p = torch.sigmoid(model(x)).item()
    return p, (p >= threshold)

# cache detector + metadata so we load once
_DET = {"model": None, "preproc": None, "stop_idx": None}

def _load_detector_once():
    if _DET["model"] is None:
        weights_det = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
        _DET["model"] = fasterrcnn_resnet50_fpn(weights=weights_det).eval()
        _DET["preproc"] = weights_det.transforms()
        _DET["stop_idx"] = weights_det.meta["categories"].index("stop sign")

def _run_detector(pil_img, thresh=0.5, draw_boxes=True):
    _load_detector_once()
    # just convert to tensor; the model does its own resize/normalize
    x = _DET["preproc"](pil_img)
    with torch.no_grad():
        out = _DET["model"]([x])[0]
    labels = out["labels"].tolist()
    scores = out["scores"].tolist()
    boxes  = out["boxes"].tolist()

    present = False
    img_to_show = pil_img.copy()
    d = ImageDraw.Draw(img_to_show) if draw_boxes else None



    # detections are sorted by score (high to low)
    for (lbl, sc, box) in zip(labels, scores, boxes):
        if lbl == _DET["stop_idx"] and sc >= thresh:
            present = True
            if draw_boxes and d is not None:
                x1, y1, x2, y2 = box
                d.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=3)
                d.text((x1+4, y1+4), f"stop sign {sc:.2f}", fill=(0, 255, 0))
    return present, img_to_show

# app to check in stop sign is present with a drag and drop
def check_stop_sign(image, mode, threshold, draw_boxes):

    if mode == "classifier":
        p, present = predict_image(image, threshold)
        msg = f"[Classifier] Stop sign prob: {p:.3f} → {'PRESENT' if present else 'NOT PRESENT'} (thr={threshold:.2f})"
        return msg, image
    else:
        present, img_out = _run_detector(image, thresh=threshold, draw_boxes=draw_boxes)
        msg = f"[Detector] Stop sign present: {present} (thr={threshold:.2f})"
        return msg, img_out

# builds the UI
demo = gr.Interface(
    fn=check_stop_sign,
    inputs=[
        gr.Image(type="pil", label="Drop an image"),
        gr.Radio(choices=["classifier", "detector"], value="classifier", label="Mode"),
        gr.Slider(0.0, 1.0, value=0.5, step=0.05, label="Threshold"),
        gr.Checkbox(value=True, label="Draw boxes (detector mode only)")
    ],
    outputs=[
        gr.Textbox(label="Result"),
        gr.Image(label="Preview")
    ],
    title="Stop Sign Presence",
    description="Use your trained classifier for a probability, or switch to the detector to visualize boxes."
)

demo.launch(share=False)


Images dir: /content/coco/val2017
Ann file: /content/coco/annotations/instances_val2017.json
loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
Counts | pos, neg, total = 69 69 138
Dataset sizes | train, val = 118 20
Device: cpu


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 01 | train 0.5926/0.636 | val 0.2395/0.950


  0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 04 | train 0.0226/1.000 | val 0.3299/0.800


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 05 | train 0.0100/1.000 | val 0.4265/0.800
Confusion matrix [TN FP; FN TP]:
[[8 1]
 [3 8]]
              precision    recall  f1-score   support

         0.0      0.727     0.889     0.800         9
         1.0      0.889     0.727     0.800        11

    accuracy                          0.800        20
   macro avg      0.808     0.808     0.800        20
weighted avg      0.816     0.800     0.800        20

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>